In [2]:
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
from IPython.display import Audio
import pandas as pd
import numpy as np 
import os
import soundfile as sf
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, auc

In [59]:
def cut_audio_file(audio_dir_test,file,start,lenght):
# to get the audio segment and save it as segment_lenght_filename.mp3
#file = filename.mp3
# start = seconds
# lenght = seconds
    filename=os.path.join(audio_dir_test,file)
    y, sr = librosa.load(filename, sr=None)
    # Define start and end times
    end_sec = start + lenght  # 4 seconds after the start
    # Convert times to samples
    start_sample = int(start * sr)
    end_sample = int(end_sec * sr)
    # Extract the segment
    segment = y[start_sample:end_sample]
    # Save the segment
    out_filename = f'extracted_segment_{lenght}_{file}'
    sf.write(os.path.join(audio_dir_test,out_filename), segment, sr)
    

In [3]:
#get all features out of giver file 
def get_features(file_path, file_name):
    y, sr = librosa.load(file_path)
    iter_ = int(y.shape[0]/sr)
    features = []
    for i in range(iter_):
        y_seg = y[i*sr:(sr*i+sr)]
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=y_seg, sr=sr))
        rms = np.mean(librosa.feature.rms(y=y_seg))
        spec_cent = np.mean(librosa.feature.spectral_centroid(y=y_seg, sr=sr))
        spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y_seg, sr=sr))
        rolloff = np.mean(librosa.feature.spectral_rolloff(y=y_seg, sr=sr))
        zcr = np.mean(librosa.feature.zero_crossing_rate(y_seg))
        mfcc = np.mean(librosa.feature.mfcc(y=y_seg, sr=sr, n_mfcc=20),axis=1)
        features.append([chroma_stft,rms,spec_cent,spec_bw,rolloff,zcr,*mfcc]) # starred expressions for mfcc to get 20 columns instead of 20 row
        # b = np.concatenate((b,a),axis=1) # ajout filename apres?
    return features
    

In [5]:
data_set = pd.read_csv('Data/DATASET-balanced.csv')


['extracted_segment_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'linus-to-musk-DEMO.mp3',
 'linus-original-DEMO.mp3']

In [20]:
def make_test_features_df(audio_dir,filename):
    file_features=get_features(os.path.join(audio_dir,file),file)
    feature_list = []
    for segment_features in file_features:
            feature_list.append(segment_features)

    features_df = pd.DataFrame(feature_list,columns = data_set.columns[0:-1])
    return features_df


In [64]:
# Start here:
# make sure it the right path
audio_dir_test='data/DEMONSTRATION/DEMONSTRATION'
files = os.listdir(audio_dir_test)
files


['.DS_Store',
 'extracted_segment_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'extracted_segment_100_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'linus-to-musk-DEMO.mp3',
 'linus-original-DEMO.mp3']

In [65]:
#select the file to cut using files[]
file_to_cut = files[3]
file_to_cut

'Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3'

In [66]:
# cut the audio, start in second, and the lenght in seconds
cut_audio_file(audio_dir_test,file_to_cut,89,10)


In [67]:
files = os.listdir(audio_dir_test)
files

['.DS_Store',
 'extracted_segment_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'extracted_segment_100_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'extracted_segment_10_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'linus-to-musk-DEMO.mp3',
 'linus-original-DEMO.mp3']

In [68]:
# select file to extract
file_toextract = files[3]

In [69]:
new_df = make_test_features_df(audio_dir_test,file_toextract)

In [70]:
new_df

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0.470935,0.059258,3589.920944,2347.309532,5931.908070,0.272317,-261.129608,74.677040,18.270967,7.238893,...,-10.181029,-0.778232,0.806979,6.729933,-0.798005,0.749729,1.258071,4.768607,-3.838733,1.802546
1,0.348168,0.078241,2023.856436,1993.776797,3844.900235,0.098311,-220.117859,111.947762,-7.605939,40.627792,...,-13.463178,2.044327,-3.555796,-0.544064,1.404545,-4.676535,-2.320460,8.780890,-2.997688,-4.718473
2,0.348726,0.069439,1200.883202,1528.429000,2143.532493,0.051991,-261.316345,162.495377,-0.482804,29.772436,...,-12.047729,2.520585,1.551374,5.591324,0.434483,-0.622147,2.380089,3.535395,-1.074344,0.098497
3,0.439243,0.035014,1953.601816,1958.732599,3729.403964,0.098722,-280.751251,131.258850,-7.916832,21.938663,...,3.202435,2.764424,-1.067935,7.193146,4.816433,4.016971,0.705184,6.899977,0.677768,-2.142981
4,0.432548,0.065496,1719.142390,1772.658174,3074.109442,0.080211,-258.324341,141.187897,-17.307430,19.037336,...,-8.798536,2.278147,0.518410,10.369609,2.086153,-3.065268,4.038606,2.917624,-3.248698,0.680514
5,0.437033,0.043098,1514.928781,1609.514980,2820.115523,0.063077,-294.809692,156.097107,-22.933355,20.246971,...,1.300806,4.573141,4.311993,11.385888,7.164671,2.683540,1.340205,10.217193,2.709661,2.730036
6,0.528886,0.028137,1535.085987,1790.471841,3090.748735,0.068448,-331.031860,138.874298,-20.648512,23.201326,...,5.077175,3.915003,4.768389,8.504746,1.135159,1.988425,-0.049400,3.882800,0.209928,-3.238527
7,0.402212,0.080992,2159.247009,1989.368595,4009.580300,0.103860,-215.871353,116.375366,-20.949364,17.507820,...,-9.070487,3.252617,2.017318,4.516620,-0.075407,1.144051,-0.266043,3.684124,0.099339,-3.277136
8,0.396729,0.063704,2069.571380,1946.995748,3934.703480,0.091431,-238.591858,126.633797,-13.024871,27.830498,...,-10.981999,1.318970,-1.697662,-0.673600,2.053971,1.183913,-0.039037,6.783774,-0.141546,0.638278
9,0.398887,0.043221,1761.637458,1831.495023,3393.192361,0.067805,-309.462219,122.819244,-13.810925,28.456377,...,-4.454094,2.786302,1.433321,4.284021,1.454062,-0.258384,3.583460,2.995882,0.792018,4.803803


In [71]:
file_toextract[:-4]

'extracted_segment_10_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs'

In [73]:
new_df.to_csv(f'data/{file_toextract[:-4]}.csv',index=False)